In [10]:
# Imports

import pandas as pd
import matplotlib.pyplot as plt
import inspect
from tqdm import tqdm
import numpy as np
import os
import tensorflow as tf
import sys
import cv2
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
import random
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import categorical_crossentropy
import shutil
import imghdr
import Augmentor
# Set batch size for training and validation
batch_size = 32

     ---------------------------------------- 4.3/4.3 MB 4.0 MB/s eta 0:00:00
  Using cached dill-0.3.5.1-py2.py3-none-any.whl (95 kB)
     ---------------------------------------- 98.1/98.1 kB 1.1 MB/s eta 0:00:00
     -------------------------------------- 51.0/51.0 kB 647.2 kB/s eta 0:00:00
  Using cached promise-2.3.tar.gz (19 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached googleapis_common_protos-1.56.3-py2.py3-none-any.whl (211 kB)
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21486 sha256=379a79c2ad66976f4e50ad2c23a105c4027e04a4590a00e4ad097669c00cc764
  Stored in directory: c:\users\rayst\appdata\local\pip\cache\wheels\e1\e8\83\ddea66100678d139b14bc87692ece57c6a2a937956d2532608
Successfully built promise


In [1]:
from pathlib import Path
import os
import sys

sys.path.append(str(Path(os.getcwd()).parents[1]))
from Helper_Functions import common_utils, classification_utils
directory_to_extract_to = os.getcwd()

common_utils.load_data_from_one_drive(directory_to_extract_to, "classification_paths", "mit_indoor_model")

Extracting : 100%|██████████| 1/1 [00:06<00:00,  6.76s/it]


In [ ]:
common_utils.load_data_from_one_drive(directory_to_extract_to, "classification_paths", "mit_indoor_data")

In [4]:
data_dir = 'content/cleaned_indoorCVPR_09_final'
Name = os.listdir(data_dir)
print(Name)
print(len(Name))


['dentaloffice', 'hairsalon', 'waitingroom']
3


In [ ]:
## Clean dataset
# os.makedirs('/content/cleaned_indoorCVPR_09')

# for i in Name:
#   source = '/content/indoorCVPR_09/Images/' + i
#   allFileNames = os.listdir(source)
#   all_FileNames = [source+'/' + name for name in allFileNames]

#   os.makedirs('/content/cleaned_indoorCVPR_09/'+i)
#   for name in all_FileNames:
#     if imghdr.what(name) == 'jpeg': # Added this line as model training was giving an error as different types of images were present in the dataset
#       shutil.copy(name, '/content/cleaned_indoorCVPR_09/' + i)

In [ ]:
# !zip -r "/content/drive/MyDrive/Data/cleaned_indoorCVPR_09_final.zip" "/content/content/cleaned_indoorCVPR_09_final"

In [5]:
import os
import numpy as np
import shutil


classes = Name

for i in Name:
  os.makedirs('train/' + i)
  os.makedirs('test/' + i)
  source = 'content/cleaned_indoorCVPR_09_final/' + i
  #source = '/improvement/cleaned_indoorCVPR_09_final/' + i
  allFileNames = os.listdir(source)
  np.random.shuffle(allFileNames)

  test_ratio = 0.05
  
  train_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                      [int(len(allFileNames)* (1 - test_ratio))])

  train_FileNames = [source+'/'+ name for name in train_FileNames.tolist()]
  test_FileNames = [source+'/' + name for name in test_FileNames.tolist()]


  for name in train_FileNames:
    shutil.copy(name, 'train/' + i)

  for name in test_FileNames:
    shutil.copy(name, 'test/' + i)

In [ ]:
## Used to recursive delete files inside specified folder
# import shutil
# shutil.rmtree('/content/train')
# shutil.rmtree('/content/test')

In [33]:
image_size = (224,224)
train_dir = 'train'
train_dataset  = tf.keras.preprocessing.image_dataset_from_directory(train_dir, 
                                                                    batch_size = 32,
                                                                    image_size = image_size,
                                                                    shuffle = True, 
                                                                    seed = 505,
                                                                    validation_split=0.1,
                                                                    subset = "training")
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(train_dir, 
                                                                    batch_size = 32,
                                                                    image_size = image_size,
                                                                    shuffle = True, 
                                                                    seed = 505,
                                                                    validation_split=0.1,
                                                                    subset = "validation")


Found 15650 files belonging to 67 classes.
Using 14085 files for training.
Found 15650 files belonging to 67 classes.
Using 1565 files for validation.


In [17]:
base_model=tf.keras.applications.EfficientNetV2B2(include_top=False, weights="imagenet",input_shape=(256, 256, 3), pooling='max') 

x=base_model.output
x=tf.keras.layers.BatchNormalization()(x)
x = Dense(224, activation='relu')(x)
x=Dropout(rate=.5)(x)        
output=Dense(len(Name), activation='softmax')(x)

model=Model(inputs=base_model.input, outputs=output)
model.compile(Adamax(learning_rate=.001), loss='sparse_categorical_crossentropy', metrics=['accuracy']) 

35848192/35839040 [==============================] - 0s 0us/step


In [18]:
#!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-decc1719-6ab1-3983-e33b-1d2ec4f90ab4)


In [ ]:
## Code to remove images
# import os

# for cls in needs_improvement_lst:
#   c=0
#   cpath = f'/content/content/cleaned_indoorCVPR_09_final/{cls}'
#   for file in os.listdir(cpath):
#         os.remove(os.path.join(cpath, file)) 

In [35]:
history=model.fit(x=train_dataset,  epochs=10, verbose=1,  validation_data=validation_dataset,
               callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=1)])

Epoch 1/10
441/441 [==============================] - 319s 688ms/step - loss: 0.2929 - accuracy: 0.9261 - val_loss: 0.1456 - val_accuracy: 0.9642
Epoch 2/10
441/441 [==============================] - 298s 674ms/step - loss: 0.1694 - accuracy: 0.9524 - val_loss: 0.1187 - val_accuracy: 0.9681
Epoch 3/10
441/441 [==============================] - 298s 676ms/step - loss: 0.1157 - accuracy: 0.9676 - val_loss: 0.1067 - val_accuracy: 0.9712
Epoch 4/10
441/441 [==============================] - 298s 675ms/step - loss: 0.0897 - accuracy: 0.9733 - val_loss: 0.0987 - val_accuracy: 0.9770
Epoch 5/10
441/441 [==============================] - 301s 682ms/step - loss: 0.0686 - accuracy: 0.9791 - val_loss: 0.0954 - val_accuracy: 0.9776
Epoch 6/10
441/441 [==============================] - 301s 681ms/step - loss: 0.0588 - accuracy: 0.9827 - val_loss: 0.0948 - val_accuracy: 0.9770


In [42]:
# model.save("best_till_now_model.h5")

In [34]:
## Load already trained model
model = tf.keras.models.load_model('my_model_latest.h5')
model.compile(Adamax(learning_rate=.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy']) 

In [36]:
Name.sort()

In [37]:
def predict (img,model):
    img = cv2.resize(img,dsize=image_size,interpolation=cv2.INTER_CUBIC)
    img = np.expand_dims(img, axis = 0)
    prediction = model.predict(img)
    cname =np.argmax(prediction)
    category = Name[cname]    
    return category

In [23]:
print(Name)

['airport_inside', 'artstudio', 'auditorium', 'bakery', 'bar', 'bathroom', 'bedroom', 'bookstore', 'bowling', 'buffet', 'casino', 'children_room', 'church_inside', 'classroom', 'cloister', 'closet', 'clothingstore', 'computerroom', 'concert_hall', 'corridor', 'deli', 'dentaloffice', 'dining_room', 'elevator', 'fastfood_restaurant', 'florist', 'gameroom', 'garage', 'greenhouse', 'grocerystore', 'gym', 'hairsalon', 'hospitalroom', 'inside_bus', 'inside_subway', 'jewelleryshop', 'kindergarden', 'kitchen', 'laboratorywet', 'laundromat', 'library', 'livingroom', 'lobby', 'locker_room', 'mall', 'meeting_room', 'movietheater', 'museum', 'nursery', 'office', 'operating_room', 'pantry', 'poolinside', 'prisoncell', 'restaurant', 'restaurant_kitchen', 'shoeshop', 'stairscase', 'studiomusic', 'subway', 'toystore', 'trainstation', 'tv_studio', 'videostore', 'waitingroom', 'warehouse', 'winecellar']


In [38]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [39]:
correct_pred = 0
total_test_images = 0
#imprv = []
for i in tqdm(Name):
  images = load_images_from_folder(f"test/{i}/")
  total_test_images+=len(images)
  #c=0
  for image in images:
    pred = predict(image, model)
    #print("predicted : ",pred, " Actual : ",i)
    if(pred == i):
      #c+=1
      correct_pred+=1
  # if (c/len(images))*100 <85:
  #   imprv.append(i)
  #   print(f'{i} : {(c/len(images))*100}')

print(f"Accuracy : {(correct_pred/total_test_images)*100}")

  4%|▍         | 3/67 [00:10<03:22,  3.17s/it]

auditorium : 83.33333333333334


  6%|▌         | 4/67 [00:12<03:04,  2.93s/it]

bakery : 63.33333333333333


  7%|▋         | 5/67 [00:15<02:51,  2.76s/it]

...

restaurant : 73.33333333333333


 84%|████████▎ | 56/67 [02:29<00:28,  2.62s/it]

restaurant_kitchen : 80.0


100%|██████████| 67/67 [02:58<00:00,  2.66s/it]

winecellar : 76.66666666666667
Accuracy : 89.25373134328358


In [ ]:
## Noticed if we resize and then data augment, data augmentation is faster
import PIL
import os
from PIL import Image


for files in tqdm(needs_improvement_lst):
  cpath = f"content/cleaned_indoorCVPR_09_final/{files}"
  for img in os.listdir(cpath):
    f1 = os.path.join(cpath, img)
    img = Image.open(f1)
    img = img.resize((256,256))
    img.save(f1)

## Code to data augment and create more images for needs_improvement_lst
for i in needs_improvement_lst:
  #os.makedirs(f'/improvement_1/cleaned_indoorCVPR_09_final/{i}')
  p = Augmentor.Pipeline(f"content/cleaned_indoorCVPR_09_final/{i}", output_directory=f'content/cleaned_indoorCVPR_09_final/{i}')
  p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=20)
  p.zoom(probability=0.5, min_factor=1.1, max_factor=1.8)
  p.flip_left_right(probability=0.7)
  p.sample(466)

In [ ]:
## code to check number of images in each folder/class
for i in tqdm(Name):
  images = load_images_from_folder(f"train/{i}/")
  print("Class : ",i,"\tNumber of images : ",len(images))

In [ ]:
## What needs to be done,

## 1) Redo random search CV code is below.
## 2) Increase image distribution for imprv lst of images. (best_till_now_model.h5 is the best model we have wit 89.5% accuracy)

In [ ]:
## Code for randomsearchCV
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
from keras.regularizers import l1_l2
from sklearn.model_selection import RandomizedSearchCV, KFold
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
import PIL
import os
from PIL import Image
import gc
gc.collect()


In [ ]:
for files in tqdm(Name):
  cpath = f"train/{files}"
  for img in os.listdir(cpath):
    f1 = os.path.join(cpath, img)
    img = Image.open(f1)
    img = img.resize((224,224))
    img.save(f1)

In [ ]:
data = []
label = []
Files = Name
label_val = 0

for files in tqdm(Files):
    cpath = f"train/{files}"
    for img in os.listdir(cpath):
        image_array = cv2.imread(os.path.join(cpath, img), cv2.IMREAD_COLOR)
        data.append(image_array)
        label.append(label_val)
    label_val = label_val + 1

In [ ]:
data = np.asarray(data)
label = np.asarray(label)

In [ ]:
def create_model( nl1=1, nl2=1,  nl3=1, 
                 nn1=1000, nn2=500, nn3 = 200, lr=0.01, l1=0.01, l2=0.01,
                act = 'relu', dropout=0):
    '''This is a model generating function so that we can search over neural net 
    parameters and architecture'''
    
    opt = Adam(lr=lr)
    #reg = l1_l2(l1=l1, l2=l2)
                                                     
    model = Sequential()
    model.add(tf.keras.applications.EfficientNetV2B2(include_top=False, weights="imagenet",input_shape=(224, 224, 3), pooling='max'))
    model.add(tf.keras.layers.BatchNormalization())
    # for the firt layer we need to specify the input dimensions
    first=True
    for i in range(nl1):
        if first:
            model.add(Dense(nn1, activation=act)
            first=False
        else: 
            model.add(Dense(nn1, activation=act)
        if dropout!=0:
            model.add(Dropout(dropout))

    for i in range(nl2):
        if first:
            model.add(Dense(nn2, activation=act)
            first=False
        else: 
            model.add(Dense(nn2, activation=act)
        if dropout!=0:
            model.add(Dropout(dropout))
            
    for i in range(nl3):
        if first:
            model.add(Dense(nn3, activation=act)
            first=False
        else: 
            model.add(Dense(nn3, activation=act)
        if dropout!=0:
            model.add(Dropout(dropout))
            
    model.add(Dense(67, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [ ]:
model = KerasClassifier(build_fn=create_model, epochs=6, batch_size=4, verbose=1)

In [ ]:
# learning algorithm parameters
lr=[1e-2, 1e-3, 1e-4]


# activation
activation=['relu', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', 'LeakyReLU']

optimizers = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']

# numbers of layers
nl1 = [0,1,2,3]
nl2 = [0,1,2,3]
nl3 = [0,1,2,3]

# neurons in each layer
nn1=[256,512,1024]
nn2=[256,512,1024]
nn3=[256,512,1024]

# dropout and regularisation
dropout = [0.4,0.5,0.6]
# l1 = [0, 0.01, 0.003, 0.001,0.0001]
# l2 = [0, 0.01, 0.003, 0.001,0.0001]

# dictionary summary
param_grid = dict(
                    nl1=nl1, nl2=nl2, nl3=nl3, nn1=nn1, nn2=nn2, nn3=nn3,
                    act=activation, lr=lr, dropout=dropout, opt = optimizers
                 )
grid = RandomizedSearchCV(estimator=model, cv=KFold(3), param_distributions=param_grid, 
                          verbose=1,  n_iter=10, n_jobs=1)



In [ ]:
grid_result = grid.fit(data, label)

In [ ]:
print(grid_result.best_params_)